## Playback an existing recording

If you don't have an EEG system currently acquiring data, you can still do testing of stimuli by playing back an existing recording. We have such a recording in example-dataset.set, whose contents are described in [2_1_SignalAcquisition.ipynb](2_1_SignalAcquisition.ipynb).

What we do in this notebook is to load in that dataset using [MNE Python](https://mne.tools/stable/index.html), examine some aspects, and then use pyLSL to create an output stream as described [here](https://github.com/sccn/labstreaminglayer/wiki/ExampleCode) where we push in that data just as if it's acquired at this very moment. Then in [2_3_SignalPlaybackCheck.ipynb](2_3_SignalPlaybackCheck.ipynb) we will examine that data.

First we grab MNE Python:
```python
import mne
```

and the dataset is here:
```python
data_file = '/home/rt/nf/example-data.set'
```

In [1]:
import mne
data_file = '/home/rt/nf/example-data.set'
data_file = '/home/johan/nf/nf-rtime/Projects_Templates/3_RTMRICWL/trial_data_for_mri_cwl_development.set'
data_file = '/home/johan/nf/nf-rtime/Projects_Templates/3_RTMRICWL/trio2_eoec_outside_before.set'

MNE Python comes with extensive tutorials how to import data and do all kinds of analyses, including spatial and temporal filtering and visualization of data; some of these things we will definitely use later on. The Python Neurofeedback toolbox made by the Russians [NFB Lab](https://github.com/nikolaims/nfb) also uses a lot of MNE's functionalities.

For now we just use the functionality to read in EEGLab data (which in our case is just a matlab.mat file, which we could also have imported with python's scipy module. But MNE makes things a little bit easier.

```python
raw = mne.io.read_raw_eeglab(data_file)
```

In [2]:
raw = mne.io.read_raw_eeglab(data_file).resample(1000)
print(raw)

<RawEEGLAB  |  trio2_eoec_outside_before.set, n_channels x n_times : 32 x 289741 (289.7 sec), ~70.8 MB, data loaded>


In [3]:
# raw.plot(scalings='auto')

So you might see some warnings about boundary, and reloading not being supported - but you also should see that there are 13 channels and 63358 datapoints, and how many seconds there are.

In [4]:
print(dir(raw))

['__class__', '__contains__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_annotations', '_cals', '_check_bad_segment', '_comp', '_data', '_dtype', '_dtype_', '_filenames', '_first_samps', '_first_time', '_get_buffer_size', '_get_channel_positions', '_init_kwargs', '_last_samps', '_last_time', '_orig_units', '_parse_get_set_params', '_pick_drop_channels', '_preload_data', '_projector', '_projectors', '_raw_extras', '_raw_lengths', '_read_comp_grade', '_read_segment', '_read_segment_file', '_set_channel_positions', '_set_dig_montage_in_init', '_size', '_times', '_update_times', 'add_channels', 'add_events', 'add_proj', 'annotati

In [5]:
# grab sample rate:
sfreq = raw.info['sfreq']
print(sfreq)

1000.0


In [6]:
# grab the number of channels
nchans = len(raw.ch_names)
print(nchans)

32


In [7]:
# grab the data as a numpy array
data = raw.get_data().astype('float32')

Numpy arrays are python's way to emulate how Matlab organizes matrices. See [here](https://numpy.org/devdocs/user/quickstart.html) for documentation on Numpy, some of the methods we're going to use below (and throughout the NF work)

numpy is usually imported into python (like matlab's addpath) like so:

```python
import numpy as np
```

we don't actually need to use np to do anything, since data is already a numpy array. MNE uses numpy inherently

In [8]:
data.shape

(32, 289741)

In [9]:
data.dtype

dtype('float32')

So now we have sfreq, nchans, and data as an np array - we will now use code found in the [Example Code Guide](https://github.com/sccn/labstreaminglayer/wiki/ExampleCode), specifically that for [python](https://github.com/labstreaminglayer/liblsl-Python/blob/master/pylsl/examples/SendData.py), to send data sample-by-sample.

The code you can find is:

```python
"""Example program to demonstrate how to send a multi-channel time series to
LSL."""

import time
from random import random as rand

from pylsl import StreamInfo, StreamOutlet

# first create a new stream info (here we set the name to BioSemi,
# the content-type to EEG, 8 channels, 100 Hz, and float-valued data) The
# last value would be the serial number of the device or some other more or
# less locally unique identifier for the stream as far as available (you
# could also omit it but interrupted connections wouldn't auto-recover)
info = StreamInfo('BioSemi', 'EEG', 8, 100, 'float32', 'myuid34234')

# next make an outlet
outlet = StreamOutlet(info)

print("now sending data...")
while True:
    # make a new random 8-channel sample; this is converted into a
    # pylsl.vectorf (the data type that is expected by push_sample)
    mysample = [rand(), rand(), rand(), rand(), rand(), rand(), rand(), rand()]
    # now send it and wait for a bit
    outlet.push_sample(mysample)
    time.sleep(0.01)
```

We will do some modifications to emulate our sampling rate (1000 Hz) in a slightly better way, since the push_sample might also take some time itself and cause some accumulation of sampling error over time for longer recordings.

In [10]:
import pylsl

In [11]:
import time

In [12]:
# create info for our purposes:
info = pylsl.StreamInfo('Playback-2', 'EEG', nchans, sfreq, 'float32', 'someidentifier123')

also, we'd need to provide LSL with some information regarding the channels as shown [here](https://github.com/labstreaminglayer/liblsl-Python/blob/master/pylsl/examples/HandleMetadata.py)

In [13]:
chns = info.desc().append_child("channels")
for label in raw.ch_names:
    ch = chns.append_child("channel")
    ch.append_child_value("label", label)
    ch.append_child_value("unit", "microvolts")
    ch.append_child_value("type", "EEG")

In [14]:
data.shape

(32, 289741)

create an outlet; this is basically the keymaster that creates/destroys tunnels (through which data flows) with incoming petitioners (clients)

Currently, no data is (yet!) flowing, but if you push data in it, it will still accumulate, like in a reservoir or a buffer.

In [15]:
import numpy as np

In [16]:
newdata = np.tile(np.sin(0.05*np.arange(data.shape[1])),(data.shape[0], 1))

In [17]:
newdata.shape

(32, 289741)

In [18]:
import matplotlib.pyplot as plt

In [19]:
# plt.plot(newdata.T[:,0])

In [20]:
outlet = pylsl.StreamOutlet(info)

In [21]:
data

array([[-1.04999335e-05, -9.06401419e-06, -1.24878543e-05, ...,
        -2.02113970e-05, -2.21330247e-05, -2.12578398e-05],
       [ 3.55000557e-05,  3.75363743e-05,  3.88596964e-05, ...,
        -3.50812734e-05, -6.98347549e-06,  1.64609319e-05],
       [-2.50000321e-05, -2.63996917e-05, -3.18689818e-05, ...,
        -3.05961221e-05, -3.44832697e-05, -3.46566485e-05],
       ...,
       [ 8.00008183e-06,  8.51854838e-06,  6.29115902e-06, ...,
         1.09856965e-05,  2.89989089e-06, -1.45635222e-05],
       [-1.00999983e-04, -9.97778116e-05, -9.92692221e-05, ...,
        -6.00128988e-05, -6.11403375e-05, -6.50213769e-05],
       [ 1.02999984e-04,  1.02501188e-04,  9.58540259e-05, ...,
         1.46237275e-04,  1.48963940e-04,  1.49035477e-04]], dtype=float32)

In [22]:
%matplotlib qt

In [23]:
raw.plot(scalings='auto');

In [24]:
plt.plot(data[0,:]);

In [25]:
# we need to transpose data, and then we can for-loop over it like so (extensively verbose):

time_to_send_new_data_point = time.time()
time_to_wait_between_sending_data_points = 1/sfreq

for data_point in data.T:
    
    time_to_send_new_data_point += time_to_wait_between_sending_data_points
    while time.time() < time_to_send_new_data_point:
        time.sleep(0.00001)
        # pass
    
    outlet.push_sample(data_point)
    

The python interpreter (behind this notebook) will 'hang' because it is executing the block of code and will continue to do so until all data has been sent out.
So now we have the stream running - you can go to [2_3_SignalPlaybackCheck.ipynb](2_3_SignalPlaybackCheck.ipynb) to take a look at it!

In [ ]:
raw_eo.info['sfreq']

In [22]:
# and we make the MNE data file from it        
raw_eo = mne.io.RawArray(newdata,
                        mne.create_info(raw.ch_names, 
                                    raw.info['sfreq'],
                                    'eeg', 
                                    None)
                       )

Creating RawArray with float64 data, n_channels=38, n_times=80000
    Range : 0 ... 79999 =      0.000 ...    79.999 secs
Ready.


In [24]:
%matplotlib qt

In [25]:
raw_eo.plot(scalings='auto');

In [26]:
%matplotlib qt